# Capstone Project - The Battle of Neighborhoods

---
## (Location-based Personalized Advertisements)

* **Introduction/Business Problem:**

Marketing is an essential topic for every business. If you want to boost your business, you have to present your products to the audience, or even better, the targeted audience. Commercials, advertisements, coupons are everywhere now in digital world nowadays. People cannot avoid them but also, they cannot be attracted to most of the information because they have limited concentration and would rather allocate more time on things they actually desire. That’s why customers are welcoming to relevant marketing messages rather than being bombarded with a plethora of deals that do not excite them. A customer in the kids’ section is likely to be interested in offers on kid garments. However, a standee displaying offers on kids’ garments is not relevant to most customers walking passed it. 

So now the question lies ahead, if you are business owner and you want to advertise your products, how can your ads be targeted more efficiently on potential customers? How can your ads be distributed to interested parties including customers with more precision and less cost if possible? These would be the business problem that we would like to solve in this project.

Geo-Targeted Mobile Ads could be a solution! Geo-targeting services from Google, Yahoo!, allow advertisers to allocate search campaign resources at a local level.  With geo-targeting options available today, audiences can be targeted at the country, state, city, and ZIP code level to determine the best potential ad placements.

* **Data:**

We mentioned above that with Geo-Targeted Mobile Ads we can target on potential customers better. Now we need to know the characteristics of each neighborhoods in the city so that we can know where to dispose different kinds of mobile Ads.

In this case, let’s use **Los Angeles** as our object. There are many great resources we can find online to grab data. We choose 'Los Angeles Times' and use the 'Mapping L.A. Boundaries API' to download neighborhood covering Los Angeles. 

(Find more info about dataset used in this project:  http://boundaries.latimes.com/set/la-county-regions-v6/)

Now we can start cleaning data and categorizing them to show neighborhoods and their coordinates. Moreover, we can combine the useful informations about venues and category from Foursquare with geolocation data processed before to have a better understanding of which region is more likely to consists of convenience store, for example. In that case we can distribute 7-11 ads or coupons to those areas where more people are interested in convenience store. Thus the money spent on ads are being used more efficiently and are bring more investment returns to business owners.

In [77]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from sklearn.cluster import DBSCAN

print('Libraries imported.')

Libraries imported.


In [2]:
!wget -q -O 'la_data.json' http://s3-us-west-2.amazonaws.com/boundaries.latimes.com/archive/1.0/boundary-set/la-county-neighborhoods-v6.geojson
print('Data downloaded!')

Data downloaded!


In [3]:
!wget -q -O 'la_data.json' https://opendata.arcgis.com/datasets/d6c55385a0e749519f238b77135eafac_0.geojson
print('Data downloaded!')

Data downloaded!


In [4]:
with open('la_data.json') as json_data:
    la_data = json.load(json_data)

Notice how all the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [5]:
neighborhoods_data = la_data['features']

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
ladata = pd.DataFrame(columns=column_names)

In [7]:
for data in neighborhoods_data:
    borough = 'los angeles'
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    
    if isinstance(neighborhood_latlon[0][0][0],list) == True:
        neighborhood_lat = neighborhood_latlon[0][0][0][1]
        neighborhood_lon = neighborhood_latlon[0][0][0][0]
    else:
        neighborhood_lat = neighborhood_latlon[0][0][1]
        neighborhood_lon = neighborhood_latlon[0][0][0]
    
    ladata = ladata.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [8]:
ladata.head()

,Borough,Neighborhood,Latitude,Longitude
0,los angeles,Adams-Normandie,34.037314,-118.300686
1,los angeles,Arleta,34.264122,-118.442546
2,los angeles,Arlington Heights,34.053060,-118.318623
3,los angeles,Atwater Village,34.153209,-118.278856
4,los angeles,Baldwin Hills/Crenshaw,34.025316,-118.361022


In [9]:
ladata = ladata.groupby('Neighborhood',as_index=False)['Latitude','Longitude'].mean()
ladata['Borough'] = 'los-angeles'
ladata = ladata[['Borough', 'Neighborhood','Latitude','Longitude']] 

In [10]:
ladata

,Borough,Neighborhood,Latitude,Longitude
0,los-angeles,Adams-Normandie,34.037314,-118.300686
1,los-angeles,Arleta,34.264122,-118.442546
2,los-angeles,Arlington Heights,34.053060,-118.318623
3,los-angeles,Atwater Village,34.153209,-118.278856
4,los-angeles,Baldwin Hills/Crenshaw,34.025316,-118.361022
5,los-angeles,Bel-Air,34.130774,-118.461647
6,los-angeles,Beverly Crest,34.079203,-118.429096
7,los-angeles,Beverly Grove,34.090261,-118.372510
8,los-angeles,Beverlywood,34.046749,-118.386092
9,los-angeles,Boyle Heights,34.060770,-118.192629


In [11]:
address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="la_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Los Angeles are 34.0536834, -118.2427669.


Create a map of Los Angeles with neighborhoods superimposed on top.

In [12]:
# create map of Los Angeles using latitude and longitude values
map_la = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(ladata['Latitude'], ladata['Longitude'], ladata['Borough'], ladata['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_la)  
    
map_la

Take an example out of LA dataset. Let's explore a random neighborhood in our dataframe.

In [21]:
CLIENT_ID = '0OXA3WJBTRQOUVC1JGKOOMCJGP1FY54HOFGH1DARHHZYBC5Q' # your Foursquare ID
CLIENT_SECRET = 'V2C4ZNK243GTOTHUHUXESUDVPURDZRTNKXKPTL5BC25MSMY1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [22]:
ladata.loc[66, 'Neighborhood']

'North Hills'

In [23]:
neighborhood_latitude = ladata.loc[66, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = ladata.loc[66, 'Longitude'] # neighborhood longitude value
neighborhood_name = ladata.loc[66, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of North Hills are 34.2202300938775, -118.495248461462.


With Foursquare, we use the latitude and longitude of neighborhood to find the surrounding venues!

Let's get the top 100 venues that are in *'North Hills'* within a radius of 500 meters.

In [24]:
LIMIT = 100 
radius = 500 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [25]:
results = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,FedEx Ship Center,Shipping Store,34.222878,-118.494636
1,Bob Hoover Jet Center,Airport Terminal,34.220155,-118.493983
2,Van Nuys Flight Center,Airport,34.220202,-118.494228
3,Jet Aviation,Airport Service,34.220318,-118.494173
4,KVNY North,Airport,34.220040,-118.494408
5,Pentastar Aviation,Airport Terminal,34.220265,-118.494070
6,Elite Helicopter Tours,Heliport,34.219556,-118.497505
7,"Lumber Liquidators, Inc.",Hardware Store,34.220968,-118.497999
8,Munchkin,Noodle House,34.222787,-118.496546
9,Walker Zanger,Furniture / Home Store,34.222803,-118.497226


In this example, we can clearly see the venues around 'North Hills' in LA. In Foursquare dataset, it returns 12 venues available that are near North Hills of Los Angeles. We have categorized those venues. Most of the venues are venues related to airport and heliport. Also there are bar, stores, gym, shipping store available. With each venues, we attached the coordinates of them and later we need these dataset to cluster similar characteristics. 

* **Methodology:**

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
la_venues = getNearbyVenues(names=ladata['Neighborhood'],
                                   latitudes=ladata['Latitude'],
                                   longitudes=ladata['Longitude']
                                  )

Adams-Normandie
Arleta
Arlington Heights
Atwater Village
Baldwin Hills/Crenshaw
Bel-Air
Beverly Crest
Beverly Grove
Beverlywood
Boyle Heights
Brentwood
Broadway-Manchester
Canoga Park
Carthay
Central-Alameda
Century City
Chatsworth
Chatsworth Reservoir
Chesterfield Square
Cheviot Hills
Chinatown
Cypress Park
Del Rey
Downtown
Eagle Rock
East Hollywood
Echo Park
El Sereno
Elysian Park
Elysian Valley
Encino
Exposition Park
Fairfax
Florence
Glassell Park
Gramercy Park
Granada Hills
Green Meadows
Griffith Park
Hancock Park
Hansen Dam
Harbor City
Harbor Gateway
Harvard Heights
Harvard Park
Highland Park
Historic South-Central
Hollywood
Hollywood Hills
Hollywood Hills West
Hyde Park
Jefferson Park
Koreatown
Lake Balboa
Lake View Terrace
Larchmont
Leimert Park
Lincoln Heights
Los Feliz
Manchester Square
Mar Vista
Mid-City
Mid-Wilshire
Mission Hills
Montecito Heights
Mount Washington
North Hills
North Hollywood
Northridge
Pacific Palisades
Pacoima
Palms
Panorama City
Pico-Robertson
Pico-Union
P

In [29]:
print(la_venues.shape)
la_venues.head()

(1442, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adams-Normandie,34.037314,-118.300686,Blu Elefant Café,34.039827,-118.303951,Café
1,Adams-Normandie,34.037314,-118.300686,Don Rigo's Tacos & Pupuseria,34.036576,-118.299249,Mexican Restaurant
2,Adams-Normandie,34.037314,-118.300686,The Wall,34.037014,-118.298325,Scenic Lookout
3,Adams-Normandie,34.037314,-118.300686,kwik-e mart,34.037119,-118.297423,Donut Shop
4,Adams-Normandie,34.037314,-118.300686,Tasty Wok,34.039729,-118.298737,Chinese Restaurant


In [30]:
print('There are {} uniques categories.'.format(len(la_venues['Venue Category'].unique())))

There are 274 uniques categories.


In [57]:
# one hot encoding
la_onehot = pd.get_dummies(la_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
la_onehot['Neighborhood'] = la_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [la_onehot.columns[-1]] + list(la_onehot.columns[:-1])
la_onehot = la_onehot[fixed_columns]

la_onehot.head()

,Zoo Exhibit,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Board Shop,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Canal,Candy Store,Caribbean Restaurant,Chinese Restaurant,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Soccer Field,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donburi Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Factory,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Light Rail Station,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Mountain,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Track,Trail,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Watch Shop,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [58]:
la_grouped = la_onehot.groupby('Neighborhood').mean().reset_index()
la_grouped.shape

(109, 274)

#### Create the new dataframe and display the top 20 venues for each neighborhood.

In [59]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [60]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = la_grouped['Neighborhood']

for ind in np.arange(la_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(la_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adams-Normandie,Café,Donut Shop,Mexican Restaurant,Pizza Place,Scenic Lookout,Chinese Restaurant,Factory,Electronics Store,Empanada Restaurant,Entertainment Service
1,Arleta,Park,Taco Place,Convenience Store,Furniture / Home Store,Baseball Field,Intersection,Music Store,Food Truck,Thai Restaurant,Film Studio
2,Arlington Heights,Korean Restaurant,Spa,Chinese Restaurant,Nail Salon,Massage Studio,Coffee Shop,Bar,Pharmacy,Intersection,Health & Beauty Service
3,Atwater Village,Women's Store,Garden,Zoo,Fast Food Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Factory,Farmers Market
4,Baldwin Hills/Crenshaw,Bakery,French Restaurant,Kitchen Supply Store,Park,American Restaurant,Electronics Store,Zoo,Farmers Market,Entertainment Service,Ethiopian Restaurant


#### We can now cluster neighborhoods into 5 clusters.

In [61]:
# set number of clusters
kclusters = 5

la_grouped_clustering = la_grouped.drop('Neighborhood',1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(la_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2],
      dtype=int32)

In [62]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

la_merged = ladata

# merge la_grouped with la_data to add latitude/longitude for each neighborhood
la_merged = la_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [63]:
la_merged = la_merged[~la_merged['Cluster Labels'].isnull()]
la_merged['Cluster Labels'] = la_merged['Cluster Labels'].astype(int)
#la_merged.head()

#### Visualize data! (About to see something interesting in the end...almost the end)

In [64]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_merged['Latitude'], la_merged['Longitude'], la_merged['Neighborhood'], la_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

At the end, we examine the clusters and determine the discriminating venue categories that distinguish each cluster.

* **Results:**

#### Cluster 1

In [65]:
la_merged.loc[la_merged['Cluster Labels'] == 0, la_merged.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Bel-Air,Scenic Lookout,Zoo,Farmers Market,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Factory,Fast Food Restaurant,Donut Shop
10,Brentwood,Food & Drink Shop,Scenic Lookout,Church,Donut Shop,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Factory,Farmers Market
27,El Sereno,Pool,Scenic Lookout,Zoo,Factory,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Farmers Market
49,Hollywood Hills West,Business Service,Scenic Lookout,Zoo,Farmers Market,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Factory,Fast Food Restaurant,Dumpling Restaurant
89,Sunland,Trail,Scenic Lookout,Zoo,Donut Shop,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Factory,Farmers Market


In [106]:
map_cluster1 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_merged.Latitude[la_merged['Cluster Labels'] == 0], la_merged.Longitude[la_merged['Cluster Labels'] == 0], la_merged.Neighborhood[la_merged['Cluster Labels'] == 0], la_merged['Cluster Labels'][la_merged['Cluster Labels'] == 0]):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[1],
        fill=True,
        fill_color=rainbow[1],
        fill_opacity=0.7).add_to(map_cluster1)
       
map_cluster1

#### Cluster 2

In [107]:
la_merged.loc[la_merged['Cluster Labels'] == 1, la_merged.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Central-Alameda,Concert Hall,Mexican Restaurant,Kitchen Supply Store,Chinese Restaurant,Art Gallery,Light Rail Station,Convenience Store,Flea Market,Fish & Chips Shop,Film Studio
57,Lincoln Heights,Food & Drink Shop,Light Rail Station,Zoo,Fast Food Restaurant,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Factory,Farmers Market,Filipino Restaurant


In [108]:
map_cluster2 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_merged.Latitude[la_merged['Cluster Labels'] == 1], la_merged.Longitude[la_merged['Cluster Labels'] == 1], la_merged.Neighborhood[la_merged['Cluster Labels'] == 1], la_merged['Cluster Labels'][la_merged['Cluster Labels'] == 1]):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[1],
        fill=True,
        fill_color=rainbow[1],
        fill_opacity=0.7).add_to(map_cluster2)
       
map_cluster2

#### Cluster 3

In [104]:
la_merged.loc[la_merged['Cluster Labels'] == 2, la_merged.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adams-Normandie,Café,Donut Shop,Mexican Restaurant,Pizza Place,Scenic Lookout,Chinese Restaurant,Factory,Electronics Store,Empanada Restaurant,Entertainment Service
1,Arleta,Park,Taco Place,Convenience Store,Furniture / Home Store,Baseball Field,Intersection,Music Store,Food Truck,Thai Restaurant,Film Studio
2,Arlington Heights,Korean Restaurant,Spa,Chinese Restaurant,Nail Salon,Massage Studio,Coffee Shop,Bar,Pharmacy,Intersection,Health & Beauty Service
3,Atwater Village,Women's Store,Garden,Zoo,Fast Food Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Factory,Farmers Market
4,Baldwin Hills/Crenshaw,Bakery,French Restaurant,Kitchen Supply Store,Park,American Restaurant,Electronics Store,Zoo,Farmers Market,Entertainment Service,Ethiopian Restaurant
6,Beverly Crest,Pool,Speakeasy,Art Museum,Zoo,Fast Food Restaurant,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Factory,Farmers Market
7,Beverly Grove,Pharmacy,Vegetarian / Vegan Restaurant,Sushi Restaurant,Restaurant,New American Restaurant,Mexican Restaurant,Japanese Restaurant,Frozen Yogurt Shop,French Restaurant,Liquor Store
8,Beverlywood,Convenience Store,Liquor Store,Recreation Center,Ice Cream Shop,Salon / Barbershop,Noodle House,Factory,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant
9,Boyle Heights,Basketball Court,Business Service,Bakery,Pizza Place,Film Studio,Zoo,Ethiopian Restaurant,Factory,Farmers Market,Fast Food Restaurant
11,Broadway-Manchester,Home Service,Coffee Shop,Mexican Restaurant,Light Rail Station,Flea Market,Fish & Chips Shop,Flower Shop,Food,Film Studio,Dumpling Restaurant


In [105]:
map_cluster3 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_merged.Latitude[la_merged['Cluster Labels'] == 2], la_merged.Longitude[la_merged['Cluster Labels'] == 2], la_merged.Neighborhood[la_merged['Cluster Labels'] == 2], la_merged['Cluster Labels'][la_merged['Cluster Labels'] == 2]):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[1],
        fill=True,
        fill_color=rainbow[1],
        fill_opacity=0.7).add_to(map_cluster3)
       
map_cluster3

In [68]:
lacluster3 = la_merged.Neighborhood[la_merged['Cluster Labels'] == 2]
la_venues3 = la_venues[la_venues.Neighborhood.isin(lacluster3)]

In [70]:
check = la_merged.loc[la_merged['Cluster Labels'] == 2, la_merged.columns[[1] + list(range(5, la_merged.shape[1]))]]
check3 = la_venues[la_venues.Neighborhood.isin(lacluster3)]
check.Neighborhood.unique().shape == check3.Neighborhood.unique().shape

True

In [71]:
check.Neighborhood.unique().shape

(99,)

In [73]:
# one hot encoding
la_onehot3 = pd.get_dummies(la_venues3[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
la_onehot3['Neighborhood'] = la_venues3['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [la_onehot3.columns[-1]] + list(la_onehot3.columns[:-1])
la_onehot3 = la_onehot3[fixed_columns]

In [74]:
la_grouped3 = la_onehot3.groupby('Neighborhood').mean().reset_index()
la_grouped3.shape

(99, 271)

In [88]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted3 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted3['Neighborhood'] = la_grouped3['Neighborhood']

for ind in np.arange(la_grouped3.shape[0]):
    neighborhoods_venues_sorted3.iloc[ind, 1:] = return_most_common_venues(la_grouped3.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted3.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Adams-Normandie,Donut Shop,Pizza Place,Scenic Lookout,Chinese Restaurant,Mexican Restaurant,Café,Film Studio,Fish & Chips Shop,Flea Market,Electronics Store,Fast Food Restaurant,Flower Shop,Farmers Market,Food,Factory,Ethiopian Restaurant,Entertainment Service,Empanada Restaurant,Filipino Restaurant,Zoo
1,Arleta,Park,Taco Place,Food Truck,Furniture / Home Store,Baseball Field,Intersection,Thai Restaurant,Music Store,Convenience Store,Fish & Chips Shop,Flea Market,Film Studio,Empanada Restaurant,Fast Food Restaurant,Flower Shop,Farmers Market,Factory,Ethiopian Restaurant,Entertainment Service,Filipino Restaurant
2,Arlington Heights,Korean Restaurant,Spa,Coffee Shop,Breakfast Spot,Nail Salon,Massage Studio,Bar,Chinese Restaurant,Pharmacy,Hot Dog Joint,Health & Beauty Service,Historic Site,Intersection,Diner,Ethiopian Restaurant,Flower Shop,Flea Market,Cycle Studio,Dance Studio,Fish & Chips Shop
3,Atwater Village,Women's Store,Garden,Zoo,Filipino Restaurant,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Factory,Farmers Market,Fast Food Restaurant,Film Studio,Dumpling Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food Court,Electronics Store,Donut Shop,Football Stadium
4,Baldwin Hills/Crenshaw,French Restaurant,American Restaurant,Bakery,Park,Kitchen Supply Store,Electronics Store,Flea Market,Fish & Chips Shop,Flower Shop,Film Studio,Empanada Restaurant,Fast Food Restaurant,Farmers Market,Factory,Food,Ethiopian Restaurant,Food Court,Food Truck,Entertainment Service,Filipino Restaurant


In [89]:
la_grouped_clustering3 = la_grouped3.drop('Neighborhood',1)

In [90]:
epsilon = 0.4
db = DBSCAN(eps=epsilon, min_samples=10, algorithm='ball_tree').fit(la_grouped_clustering3)
cluster_labels = db.labels_
list(cluster_labels).count(0)

66

In [91]:
# add clustering labels
neighborhoods_venues_sorted3.insert(0, 'Cluster Labels', cluster_labels)

la_merged3 = ladata

# merge la_grouped with la_data to add latitude/longitude for each neighborhood
la_merged3 = la_merged3.join(neighborhoods_venues_sorted3.set_index('Neighborhood'), on='Neighborhood')

In [93]:
la_merged3 = la_merged3[~la_merged3['Cluster Labels'].isnull()]
la_merged3['Cluster Labels'] = la_merged3['Cluster Labels'].astype(int)
#la_merged.head()

In [94]:
# create map
map_clusters3 = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_merged3['Latitude'], la_merged3['Longitude'], la_merged3['Neighborhood'], la_merged3['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters3)
       
map_clusters3

In [95]:
la_merged3.loc[la_merged3['Cluster Labels'] == 0, la_merged3.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adams-Normandie,Donut Shop,Pizza Place,Scenic Lookout,Chinese Restaurant,Mexican Restaurant,Café,Film Studio,Fish & Chips Shop,Flea Market,Electronics Store
1,Arleta,Park,Taco Place,Food Truck,Furniture / Home Store,Baseball Field,Intersection,Thai Restaurant,Music Store,Convenience Store,Fish & Chips Shop
2,Arlington Heights,Korean Restaurant,Spa,Coffee Shop,Breakfast Spot,Nail Salon,Massage Studio,Bar,Chinese Restaurant,Pharmacy,Hot Dog Joint
7,Beverly Grove,Pharmacy,New American Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Restaurant,Mexican Restaurant,Donut Shop,Café,Pet Store,Liquor Store
8,Beverlywood,Salon / Barbershop,Recreation Center,Ice Cream Shop,Convenience Store,Liquor Store,Noodle House,Zoo,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant
12,Canoga Park,Coffee Shop,Burger Joint,Gym / Fitness Center,Martial Arts Dojo,Gymnastics Gym,Shoe Store,Business Service,Burrito Place,Gym,Donburi Restaurant
13,Carthay,Grocery Store,Pharmacy,Food Truck,Laundromat,Coffee Shop,Café,Bakery,Bus Stop,Fish & Chips Shop,Ethiopian Restaurant
15,Century City,Coffee Shop,Hot Dog Joint,Farmers Market,Hotel,Business Service,BBQ Joint,Boutique,Market,Pharmacy,Steakhouse
18,Chesterfield Square,Fast Food Restaurant,American Restaurant,Bakery,Bus Station,Health & Beauty Service,BBQ Joint,Seafood Restaurant,Home Service,Film Studio,Fish & Chips Shop
20,Chinatown,Art Gallery,Theater,Deli / Bodega,Music Venue,Gym,Athletics & Sports,Asian Restaurant,Speakeasy,Women's Store,Winery


In [97]:
la_merged3.loc[la_merged3['Cluster Labels'] == -1, la_merged3.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Atwater Village,Women's Store,Garden,Zoo,Filipino Restaurant,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Factory,Farmers Market,Fast Food Restaurant
4,Baldwin Hills/Crenshaw,French Restaurant,American Restaurant,Bakery,Park,Kitchen Supply Store,Electronics Store,Flea Market,Fish & Chips Shop,Flower Shop,Film Studio
6,Beverly Crest,Pool,Speakeasy,Art Museum,Zoo,Filipino Restaurant,Entertainment Service,Ethiopian Restaurant,Factory,Farmers Market,Fast Food Restaurant
9,Boyle Heights,Basketball Court,Business Service,Bakery,Film Studio,Pizza Place,Filipino Restaurant,Ethiopian Restaurant,Factory,Farmers Market,Fast Food Restaurant
11,Broadway-Manchester,Mexican Restaurant,Home Service,Coffee Shop,Light Rail Station,Food,Flower Shop,Flea Market,Food Court,Fish & Chips Shop,Electronics Store
19,Cheviot Hills,Paper / Office Supplies Store,Food,Dumpling Restaurant,Food Court,Flower Shop,Flea Market,Fish & Chips Shop,Film Studio,Filipino Restaurant,Fast Food Restaurant
28,Elysian Park,Spiritual Center,Rental Car Location,Burrito Place,Trail,Seafood Restaurant,Rock Club,Big Box Store,Fish & Chips Shop,Film Studio,Filipino Restaurant
30,Encino,Baseball Stadium,Dog Run,Baseball Field,Entertainment Service,Factory,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Zoo
33,Florence,Health Food Store,Food,Burger Joint,Taco Place,Grocery Store,Farmers Market,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Factory
34,Glassell Park,Grocery Store,Music Venue,Intersection,Filipino Restaurant,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Factory,Farmers Market,Fast Food Restaurant


In [99]:
map_cluster31 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_merged3.Latitude[la_merged3['Cluster Labels'] == 0], la_merged3.Longitude[la_merged3['Cluster Labels'] == 0], la_merged3.Neighborhood[la_merged3['Cluster Labels'] == 0], la_merged3['Cluster Labels'][la_merged3['Cluster Labels'] == 0]):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[1],
        fill=True,
        fill_color=rainbow[1],
        fill_opacity=0.7).add_to(map_cluster31)
       
map_cluster31

#### Cluster 4

In [100]:
la_merged.loc[la_merged['Cluster Labels'] == 3, la_merged.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
82,Sepulveda Basin,General Entertainment,Zoo,Farmers Market,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Factory,Fast Food Restaurant,Food Court


In [101]:
map_cluster4 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_merged.Latitude[la_merged['Cluster Labels'] == 3], la_merged.Longitude[la_merged['Cluster Labels'] == 3], la_merged.Neighborhood[la_merged['Cluster Labels'] == 3], la_merged['Cluster Labels'][la_merged['Cluster Labels'] == 3]):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[1],
        fill=True,
        fill_color=rainbow[1],
        fill_opacity=0.7).add_to(map_cluster4)
       
map_cluster4

#### Cluster 5

In [102]:
la_merged.loc[la_merged['Cluster Labels'] == 4, la_merged.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,Granada Hills,Intersection,Filipino Restaurant,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Factory,Farmers Market,Fast Food Restaurant,Zoo,Dumpling Restaurant
63,Mission Hills,Park,Intersection,Zoo,Farmers Market,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Factory,Fast Food Restaurant


In [103]:
map_cluster5 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_merged.Latitude[la_merged['Cluster Labels'] == 4], la_merged.Longitude[la_merged['Cluster Labels'] == 4], la_merged.Neighborhood[la_merged['Cluster Labels'] == 4], la_merged['Cluster Labels'][la_merged['Cluster Labels'] == 4]):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[1],
        fill=True,
        fill_color=rainbow[1],
        fill_opacity=0.7).add_to(map_cluster5)
       
map_cluster5